In [1]:
#|default_exp test

In [2]:
#|export
from fastcore.all import *
from nbprocess.read import *
import time
import os, sys
import traceback

# Test Notebooks
> Run unit tests on notebooks in parallel

In [3]:
#|export
def nbglob(fname=None, recursive=None, config_key='nbs_path') -> L:
    "Find all files in a directory matching an extension given a `config_key`."
    if recursive is None: recursive=get_config().get('recursive', 'False').lower() == 'true'
    fname = Path(fname or get_config().path(config_key))
    return globtastic(path=fname,recursive=recursive,file_glob='*.ipynb', skip_file_re='^[_.]', skip_folder_re='^[_.]')

In [4]:
#|export
_re_directives = re.compile(r'^\s*#\s*\|\s*(.*)', flags=re.MULTILINE)

In [5]:
#|hide
_test="""
#| foo
 #| bar
 #|baz
#|eval: false
"""

test_eq(_re_directives.findall(_test), ['foo', 'bar', 'baz', 'eval: false'])

In [6]:
#|export
def _is_intersect(l1, l2): return bool(set(L(l1)).intersection(set(L(l2))))

In [7]:
#|hide
assert _is_intersect([1,2,3], 3)
assert not _is_intersect([1,2,3], ['3'])

In [8]:
#|export
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [9]:
#|export
def test_nb(fn, skip_flags=None, force_flags=None):
    "Execute tests in notebook in `fn` except those with `skip_flags`"
    os.environ["IN_TEST"] = '1'
    flags=set(L(skip_flags)) - set(L(force_flags))
    rnr = NBRunner()
    print(f"testing {fn}")
    start = time.time()
    try:
        nb = read_nb(fn)
        nbcode = nb.cells.filter(lambda x: x.cell_type == 'code' and x.source)
        for cell in nbcode:
            directives = _re_directives.findall(cell.source)
            if not _is_intersect(flags, directives):
                with HiddenPrints():
                    rnr.run(cell)
        return True,time.time()-start
    except Exception as e:
        fence = '='*50
        print(f'Error in {fn} when running:\n{fence}\n{cell.source}\n\n{type(e).__name__}: {e}\n{fence}')
        traceback.print_exc()
        return False,time.time()-start
    finally: 
        if "IN_TEST" in os.environ:
            os.environ.pop("IN_TEST")

`test_nb` can test a notebook, and skip over certain flags:

In [10]:
_nb = Path('../tests/directives.ipynb')
assert test_nb(_nb, skip_flags=['notest', 'eval: false'])[0]

testing ../tests/directives.ipynb


Sometimes you may wish to override one or more of the skip_flags, in which case you can use the argument `force_flags` which will remove the appropriate tag(s) from `skip_flags`.  This is useful because `skip_flags` are meant to be set in the `tst_flags` field of `settings.ini`, whereas `skip_flags` are usually passed in by the user.

In [11]:
#|notest
assert not test_nb(_nb, skip_flags=['notest', 'eval: false'], force_flags=['notest'])[0]

testing ../tests/directives.ipynb
Error in ../tests/directives.ipynb when running:
#| notest
assert 1 == 2

AssertionError: 


Traceback (most recent call last):
  File "/var/folders/jj/xl1rktcs6mn7ms6b8vvx8k4r0000gn/T/ipykernel_44096/2125186068.py", line 16, in test_nb
    rnr.run(cell)
  File "/Users/hamel/github/nbprocess/nbprocess/read.py", line 68, in run
    res = self(cell.source)
  File "/Users/hamel/opt/anaconda3/lib/python3.9/site-packages/tinykernel/__init__.py", line 19, in __call__
    self._run(p, nm)
  File "/Users/hamel/opt/anaconda3/lib/python3.9/site-packages/tinykernel/__init__.py", line 12, in _run
    def _run(self, p, nm, mode='exec'): return eval(compiler(p, nm, mode), self.glb)
  File "<kernel-1-0b19672f9559>", line 2, in <module>
    assert 1 == 2
AssertionError


In [16]:
#|export
@call_parse
def nbprocess_test(
    fname:str=None,  # A notebook name or glob to convert
    flags:str=None,  # Space separated list of test flags you want to run that are normally ignored
    n_workers:int=None,  # Number of workers to use
    timing:bool=False,  # Timing each notebook to see the ones are slow
    pause:float=0.5  # Pause time (in secs) between notebooks to avoid race conditions
):
    "Test in parallel the notebooks matching `fname`, passing along `flags`"
    skip_flags = get_config().get('tst_flags')
    skip_flags = (skip_flags.split() if skip_flags else []) + ['eval: false']
    force_flags = flags.split() if flags else []
    files = nbglob(fname)
    files = [Path(f).absolute() for f in sorted(files)]
    assert len(files) > 0, "No files to test found."
    if n_workers is None: n_workers = 0 if len(files)==1 else min(num_cpus(), 8)
    # make sure we are inside the notebook folder of the project
    os.chdir(get_config().path("nbs_path"))
    results = parallel(test_nb, files, skip_flags=skip_flags, force_flags=force_flags, n_workers=n_workers, pause=pause)
    passed,times = [r[0] for r in results],[r[1] for r in results]
    if all(passed): print("All tests are passing!")
    else:
        msg = "The following notebooks failed:\n"
        raise Exception(msg + '\n'.join([f.name for p,f in zip(passed,files) if not p]))
    if timing:
        for i,t in sorted(enumerate(times), key=lambda o:o[1], reverse=True):
            print(f"Notebook {files[i].name} took {int(t)} seconds")

In [14]:
#|notest
nbprocess_test()

testing /Users/hamel/github/nbprocess/nbs/01_read.ipynb
testing /Users/hamel/github/nbprocess/nbs/02_maker.ipynb
testing /Users/hamel/github/nbprocess/nbs/03_process.ipynb
testing /Users/hamel/github/nbprocess/nbs/04a_export.ipynb
testing /Users/hamel/github/nbprocess/nbs/04b_doclinks.ipynb
testing /Users/hamel/github/nbprocess/nbs/05_sync.ipynb
testing /Users/hamel/github/nbprocess/nbs/06_merge.ipynb
testing /Users/hamel/github/nbprocess/nbs/08_showdoc.ipynb
testing /Users/hamel/github/nbprocess/nbs/09_processors.ipynb
testing /Users/hamel/github/nbprocess/nbs/10_cli.ipynb
testing /Users/hamel/github/nbprocess/nbs/11_clean.ipynb
testing /Users/hamel/github/nbprocess/nbs/13_lookup.ipynb
testing /Users/hamel/github/nbprocess/nbs/14_test.ipynb
testing /Users/hamel/github/nbprocess/nbs/index.ipynb
All tests are passing!


In [15]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()